In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder

/home/dmitry/.pyenv/versions/3.6.4/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


# Градиентный бустинг
Как и бэггинг, градиентный бустинг использует ансамбль слабых моделей чтобы получить сильную. Однако, есть пара важных отличий:
1. При бустинге среди моделей проводится *взвешенное* голосование. В бэггинге веса не используются, все модели считаются равноправными.
1. При бустинге ансамбль собирается *последовательно* и каждая следующая модель пытается учесть недостатки предыдущей

Как добиться последнего? Базовые алгоритмы (составляющие ансамбль) на шаге T будут строиться путём минимизирования функционала качества по $\alpha$ и $b$:

$$ Q(\alpha, b) = \sum_{i=1}^{l} \mathcal{L}(\sum_{t=1}^{T-1}\alpha_t b_t(x_i) + \alpha b(x_i), y_i)), (1)$$
где $\mathcal{L}$ - некоторая функция потерь.

Однако, такая оптимизационная задача крайне нетривиальна. Применим тактическую хитрость. Давайте рассмотрим вектор ответов алгоритма на шаге $T-1$ на всех примерах обучающей выборки: 

$$u_{T-1} = (b(x_1), b(x_2), ... , b(x_l)), b(x) = \sum_{t=1}^{T-1}\alpha_t b_t(x)$$

И рассмотреть эту задачу как $Q(u) = \mathcal{L}(u, y) \rightarrow \min$. Градиентный метод оптимизации для решения этой задачи выглядит так:

$$u_{T, i} = u_{T-1, i} - \alpha g_i, (2)$$

где $g_i = \mathcal{L}^\prime(u_{T-1, i}, y)$ - компоненты вектора градиента.


А теперь самый важный момент - если взглянуть на формулы (1) и (2), то несложно понять, если $u_{T-1, i}$ это ни что иное как $\sum_{t=1}^{T-1}\alpha_t b_t(x)$, то выбор нового базового алгоритма $b(x_i)$ в (1) можно заменить градиентным шагом $g_i$ из (2).

Иными словами, на каждом шаге будем искать такой базовый алгоритм $b_T$, чтобы вектор $u_{T} = (b_T(x_1), b_T(x_2), ... , b_T(x_l))$ приближал бы вектор антиградиента $-\vec{g}$

## Реализация в sklearn
Разумеется, всё это уже реализовано до нас. Воспользуемся реализацией градиентного бустинга из библиотеки `sklearn.ensemble`. Там он лежит под именем `GradientBoostingRegressor` и создаётся примерно так же, как и `RandomForest`.

In [2]:
titanic = sns.load_dataset('titanic')
titanic = titanic.drop(columns=['class', 'who', 'adult_male', 'embark_town', 'alive', 'alone'])
titanic['deck'] = titanic['deck'].astype('str')
titanic.loc[titanic['deck'].eq('nan'), 'deck'] = 'U'

titanic['embarked'] = titanic['embarked'].astype('str')
titanic.loc[titanic['embarked'].eq('nan'), 'embarked'] = 'U'

titanic = titanic.dropna(subset=['age'])

encoders = {}
for column in titanic.columns:
    if titanic[column].dtype == 'object':
        encoder = LabelEncoder()
        encoder.fit(titanic[column])
        titanic[column] = encoder.transform(titanic[column])
        encoders[column] = encoder

In [3]:
def predict_by_booster(X_train, y_train, X_test, y_test):
    tree = GradientBoostingClassifier(loss='deviance', 
                                      learning_rate=0.1,
                                      n_estimators=10, 
                                      criterion='friedman_mse', 
                                      random_state=42)
    tree.fit(X_train, y_train)
    predict = tree.predict(X_test)

    acc = np.abs(predict == y_test).mean()
    return acc

titanic = titanic.sample(frac=1, replace=False, random_state=42)
X = titanic.drop(columns=['survived'])
y = titanic['survived'].copy()

splits = np.linspace(0, y.size, 6)
splits = splits.astype(int)

acc = []

for hi, low in zip(splits[:-1], splits[1:]):
    train_mask = np.ones_like(y.values, dtype=bool)
    train_mask[hi:low] = 0
    test_mask = ~train_mask
    
    X_train, X_test = X[train_mask], X[test_mask]
    y_train, y_test = y[train_mask], y[test_mask]
    
    acc.append(predict_by_booster(X_train, y_train, X_test, y_test))

print(np.mean(acc))


0.8053186250369349


https://colab.research.google.com/drive/1nXOlrmVHqCHiixqiMF6H8LSciz583_W2#scrollTo=g7s7Grj-A-Sf

## Реализации градиентного бустинга